In [1]:
# Modules
import geemap
import ee
import numpy
import matplotlib.pyplot as plt
import folium
import IPython.display as disp

In [2]:
# Authenticate Earth Engine
ee.Authenticate()

Enter verification code:  4/1AfJohXmW3QeRLSSueXyV0bf9pD9EMsS1rZU4lY95kcdbSl1HkGGQxmMUH0w



Successfully saved authorization token.


In [3]:
# Initiate the library
ee.Initialize()

In [4]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [5]:
Sentinel_trial = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [6]:
Region = ee.Geometry.Rectangle(130.9825040586519,-12.591236530819916, 130.80603616314409,-12.482652274143653)

In [7]:
Dataset_db = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                       .filterBounds(Region)
                       .filterDate(ee.Date('2023-09-01'), ee.Date('2023-09-02'))
                       .first()
                       .clip(Region))
Dataset_fl = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                       .filterBounds(Region)
                       .filterDate(ee.Date('2023-09-01'), ee.Date('2023-09-02'))
                       .first()
                       .clip(Region))

In [8]:
Dataset_db.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60']

In [9]:
url = Dataset_db.select('B11').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [10]:
location = Region.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(Dataset_db.select('B12'),
                   Dataset_db.select('B11'),
                   Dataset_db.select('B11').divide(Dataset_db.select('B12')))

# Create the map object.
m = folium.Map(location=location, zoom_start=13)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

# **Cloud Masking**

In [11]:
# Date Validate
Start_Date = '2023-09-01'
End_Date = '2023-09-02'

In [12]:
# Cloud masking
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [13]:
def Get_S2_SR_Cloud_Col(Region, start_date, end_date):
    # Import and filter S2 SR.
    S2_SR = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(Region)
        .filterDate(Start_Date, End_Date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    S2_Cloudless = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(Region)
        .filterDate(Start_Date, End_Date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': S2_SR,
        'secondary': S2_Cloudless,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [14]:
S2_SR_Cloud_Col_Eval = Get_S2_SR_Cloud_Col(Region, Start_Date, End_Date)

In [15]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [16]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [17]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [18]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [19]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = Region.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [20]:
S2_SR_Cloud_Col_Eval_Disp = S2_SR_Cloud_Col_Eval.map(add_cld_shdw_mask)

display_cloud_layers(S2_SR_Cloud_Col_Eval_Disp)

# **Remove Masking**

In [21]:
Region = ee.Geometry.Rectangle(130.9825040586519,-12.591236530819916, 130.80603616314409,-12.482652274143653)
START_DATE = '2023-09-01'
END_DATE = '2023-09-02'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

In [22]:
S2_SR_Cloud_Col = Get_S2_SR_Cloud_Col(Region, START_DATE, END_DATE)

In [23]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [24]:
S2_SR_Median = (S2_SR_Cloud_Col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [25]:
# Create a folium map object.
center = Region.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(S2_SR_Median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

# **Atmospheric Correction**

In [26]:
#from google.colab import drive
#drive.mount('/content/drive')

In [27]:
#https://github.com/ridhodwid/gee-atmcorr-S2/blob/kaggle-usage/bin/atmospheric.py

import requests

url = "https://github.com/ridhodwid/gee-atmcorr-S2/blob/kaggle-usage/bin/atmospheric.py"
response = requests.get(url)

with open("filename", "wb") as file:
    file.write(response.content)


In [28]:
# Import Modules

from Py6S import *
import datetime
import math
import os
import sys
sys.path.append('/content/drive/MyDrive/')
from atmospheric import Atmospheric

In [29]:
SixS

Py6S.sixs.SixS

In [30]:
# Mark the Region of Interest
ROI= ee.Geometry.Rectangle(130.9825040586519,-12.591236530819916, 130.80603616314409,-12.482652274143653)

In [31]:
# The first Sentinel 2 image
S2_TOA = ee.Image(
  ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(ROI)
    .filterDate('2023-09-01', '2023-09-02')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
  )

# top of atmosphere reflectance
TOA = S2_TOA.divide(10000)
display(S2_TOA)

In [32]:
info = S2_TOA.getInfo()['properties']
scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']
date = ee.Date(scene_date.strftime("%Y-%m-%d"))

In [33]:
h2o = Atmospheric.water(ROI,date).getInfo()
o3 = Atmospheric.ozone(ROI,date).getInfo()
aot = Atmospheric.aerosol(ROI,date).getInfo()
display(h2o,o3,aot)

1.8299999237060547

0.274

0.22833332419395447

In [34]:
# Calculate target altitude in km
SRTM = ee.Image('CGIAR/SRTM90_V4') # Shuttle Radar Topography mission covers *most* of the Earth
alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = ROI.centroid()).get('elevation').getInfo()
km = alt/1000 # i.e. Py6S uses units of kilometers
display(SRTM,alt,km)

Name,Description
elevation,Elevation


9

0.009

In [35]:
# Instantiate
s = SixS()

# Atmospheric constituents
s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
s.aero_profile = AeroProfile.Continental
s.aot550 = aot

# Earth-Sun-satellite geometry
s.geometry = Geometry.User()
s.geometry.view_z = 0               # always NADIR (I think..)
s.geometry.solar_z = solar_z        # solar zenith angle
s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
s.altitudes.set_sensor_satellite_level()
s.altitudes.set_target_custom_altitude(km)

In [36]:
# Spectral Response Functions
# To obtain the spectral response fx of each band

def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """

    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }

    return Wavelength(bandSelect[bandname])

In [37]:
# TOA Reflectance to Radiance
def TOA_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """

    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))

    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4)) # From http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year

    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)

    # at-sensor radiance
    rad = TOA.select(bandname).multiply(multiplier)

    return rad

In [38]:
# Radiance to Surface Reflectance
def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """

    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()

    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity

    # radiance to surface reflectance
    rad = TOA_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))


    return ref

In [39]:
# Use this if you only need surface reflectance of rgb channel
b = surface_reflectance('B8')
g = surface_reflectance('B11')
r = surface_reflectance('B12')
ref = r.addBands(g).addBands(b)

# Calculate surface reflectance for all wavebands
output = S2_TOA.select('QA60')
for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
    print(band)
    band_reflectance = surface_reflectance(band)
    output = output.addBands(surface_reflectance(band))

B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12


In [40]:
# select band of interest
band11 = ref.select('B11')
band12 = ref.select('B12')

display(band11,band12)

In [41]:
# Create a folium map object.
center = ROI.centroid(10).coordinates().reverse().getInfo()
S2_AC = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
S2_AC.add_ee_layer(band11,
                {'min': 0, 'max': 0.3, 'gamma': 3},
                'S2_Band11', True, 1, 9)

# Add layers to the folium map.
S2_AC.add_ee_layer(band12,
                {'min': 0, 'max': 0.3, 'gamma': 3},
                'S2_Band12',True, 1, 9)

# Add a layer control panel to the map.
S2_AC.add_child(folium.LayerControl())

# Display the map.
display(S2_AC)

# **Save Image as Tif by Exporting to Drive**

In [42]:
# Get the folium map as an image
image = geemap.Map.to_image(S2_AC)

# Display the image
print("Captured Image:", image)
display(image)

Captured Image: None


None

In [43]:
# Export the clipped image to Google Drive as a GeoTIFF file
export_params = {
    'image': S2_TOA.select(['B11', 'B12']),
    'description': 'Sentinel_AC',
    'scale': 10,
    'region': ROI,
    'fileFormat': 'GeoTIFF',  # Specify the file format
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

In [44]:
task.status()

{'state': 'READY',
 'description': 'Sentinel_AC',
 'creation_timestamp_ms': 1700793382708,
 'update_timestamp_ms': 1700793382708,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'AFK6RUHAX4YNYT4G4K4SGOST',
 'name': 'projects/earthengine-legacy/operations/AFK6RUHAX4YNYT4G4K4SGOST'}

**EXPORT TO GOOGLE EARTH ENGINE**

In [45]:
# # set some properties for export
dateString = scene_date.strftime("%Y-%m-%d")
ref = ref.set({'satellite':'Sentinel 2',
              'fileID':info['system:index'],
              'date':dateString,
              'aerosol_optical_thickness':aot,
              'water_vapour':h2o,
              'ozone':o3})

In [46]:
# Define YOUR Google Earth Engine assetID
# in my case it was something like this..
assetID = 'users/Hareeshrao-LC60/Atmospheric_Correction' +dateString

In [47]:
# Define the region using a polygon or coordinates
region = ee.Geometry.Polygon(
    [[
        [130.9825040586519, -12.482652274143653],
        [130.9825040586519, -12.591236530819916],
        [130.80603616314409, -12.591236530819916],
        [130.80603616314409, -12.482652274143653]
    ]])

# Export
export = ee.batch.Export.image.toAsset(\
    image=ref,
    description='sentinel2_atmcorr_export',
    assetId = assetID,
    region = region)

# uncomment to run the export
export.start()

# **Extract Pixel Values**

In [48]:
import rasterio
import pandas as pd

# Replace 'your_exported_image.tif' with the actual filename
tiff_file_path = 'C:/Users/haree/Desktop/Methane/Sentinel_AC.tif'

# Open the GeoTIFF file using rasterio
with rasterio.open(tiff_file_path) as src:
    # Read the entire image as a NumPy array
    image_array = src.read()
    
    # Get spatial information
    transform = src.transform
    metadata = src.meta

# Get image dimensions (rows, columns, bands)
rows, cols, bands = image_array.shape

# Reshape the array for easy conversion to DataFrame
reshaped_array = image_array.reshape((rows * cols, bands))

# Create a DataFrame from the reshaped array
df = pd.DataFrame(reshaped_array, columns=[f'Band_{i+1}' for i in range(bands)])

# Add latitude and longitude columns
df['Latitude'], df['Longitude'] = zip(*[transform * (col, row) for row in range(rows) for col in range(cols)])

# Display the array (for demonstration purposes)
print("Pixel Values:")
print(image_array)

Pixel Values:
[[[   0    0    0 ... 3058 3058 2782]
  [   0    0    0 ... 2556 2556 2270]
  [   0    0    0 ... 2556 2556 2270]
  ...
  [   0    0    0 ... 2484 2484 2144]
  [   0    0    0 ... 2484 2484 2144]
  [   0    0    0 ... 2364 2364 2542]]

 [[   0    0    0 ... 2255 2255 2279]
  [   0    0    0 ... 1834 1834 1478]
  [   0    0    0 ... 1834 1834 1478]
  ...
  [   0    0    0 ... 1383 1383 1280]
  [   0    0    0 ... 1383 1383 1280]
  [   0    0    0 ... 1304 1304 1440]]]


In [49]:
# Export DataFrame to CSV
csv_file_path = 'pixel_values.csv'
df.to_csv(csv_file_path, index=True)

print(f"CSV file '{csv_file_path}' exported successfully.")

CSV file 'pixel_values.csv' exported successfully.


# **Plot Pixel Values Graph**

In [50]:
with rasterio.open(tiff_file_path) as src:
    # Get metadata
    metadata = src.meta

    # Get CRS
    crs = src.crs

print("Metadata:", metadata)
print("CRS:", crs)


Metadata: {'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 1928, 'height': 1216, 'count': 2, 'crs': CRS.from_epsg(32752), 'transform': Affine(10.0, 0.0, 696190.0,
       0.0, -10.0, 8619410.0)}
CRS: EPSG:32752


In [51]:
from affine import Affine

# Extracted from your metadata
transform = Affine(10.0, 0.0, 696190.0, 0.0, -10.0, 8619410.0)

# Calculate the center coordinates of the pixel at row i, column j
i = 500  # Replace with your desired row index
j = 800  # Replace with your desired column index

x = transform[0] + j * transform[1] + i * transform[2]
y = transform[3] + j * transform[4] + i * transform[5]

print("X coordinate:", x)
print("Y coordinate:", y)


X coordinate: 348095010.0
Y coordinate: 4309697000.0


In [52]:
# Install modules
!pip install pyproj
!pip install pyepsg

# Transform CRS to Lon, Lat
from pyproj import CRS, Transformer

source_crs = CRS.from_epsg(32752)
target_crs = CRS.from_epsg(4326) #Based on World Geodetic System (WGS84)

transformer = Transformer.from_crs(source_crs, target_crs)

#Define x and y
x = 348095010.0
y = 4309697000.0

lon, lat = transformer.transform(x, y)

# Print the transformed coordinates
print("Longitude:", lon)
print("Latitude:", lat)

Longitude: inf
Latitude: inf


In [53]:

import rioxarray

# Replace 'your_exported_image.tif' with the actual filename
tiff_file_path = 'C:/Users/haree/Desktop/Methane/Sentinel_AC.tif'
rds = rioxarray.open_rasterio(tiff_file_path)
rds_4326 = rds.rio.reproject("EPSG:4326")
rds_4326.rio.to_raster("file.tif")

rds_4326.rio.to_raster("file.tif", compress="DEFLATE", tiled=True)

In [54]:
import ee
import geemap
import folium


# Function to mask clouds using the Sentinel-2 QA band

# Function to calculate methane concentration from Sentinel-2 image
def calculate_methane_concentration(image):
    # Replace 'your_band_name' with the actual band containing atmospheric correction results
    methane_band = image.select('B12')
    # Specify the conversion factor if needed
    # methane_concentration = methane_band.multiply(CONVERSION_FACTOR)
    return methane_band

# Replace 'your_sentinel_image' with the actual Sentinel image
sentinel_image = ee.Image(S2_TOA)

# Apply atmospheric correction if needed
#atmospheric_corrected_image = sentinel_image.some_atmospheric_correction_function()

# Mask clouds
#masked_image = mask_clouds(atmospheric_corrected_image)

# Calculate methane concentration
methane_concentration_image = calculate_methane_concentration(sentinel_image)

# Display the methane concentration on the Folium map
Map = geemap.Map()
Map.centerObject(methane_concentration_image, zoom=10)
Map.addLayer(methane_concentration_image, {
    'bands': ['B12'],
    'min': 0,
    'max': 200  # Adjust the min and max values as needed
}, 'Methane Concentration')
Map.addLayerControl()
Map

# You can now visualize the methane concentration on the Folium map and further analyze the data.


Map(center=[-12.235938770667586, 131.32795419893583], controls=(WidgetControl(options=['position', 'transparen…

In [55]:
import ee
import geemap
import folium


# Function to mask clouds using the Sentinel-2 QA band

# Function to calculate methane concentration from Sentinel-2 image
def calculate_methane_concentration(image):
    # Replace 'your_band_name' with the actual band containing atmospheric correction results
    methane_band = image.select('B8')
    # Specify the conversion factor if needed
    # methane_concentration = methane_band.multiply(CONVERSION_FACTOR)
    return methane_band

# Replace 'your_sentinel_image' with the actual Sentinel image
sentinel_image = ee.Image(S2_TOA)

# Apply atmospheric correction if needed
#atmospheric_corrected_image = sentinel_image.some_atmospheric_correction_function()

# Mask clouds
#masked_image = mask_clouds(atmospheric_corrected_image)

# Calculate methane concentration
methane_concentration_image = calculate_methane_concentration(sentinel_image)

# Display the methane concentration on the Folium map
Map = geemap.Map()
Map.centerObject(methane_concentration_image, zoom=10)
Map.addLayer(methane_concentration_image, {
    'bands': ['B8'],
    'min': 0,
    'max': 200  # Adjust the min and max values as needed
}, 'Methane Concentration')
Map.addLayerControl()
Map

# You can now visualize the methane concentration on the Folium map and further analyze the data.


Map(center=[-12.235938770667586, 131.32795419893583], controls=(WidgetControl(options=['position', 'transparen…

# **Plume Transmission Ratio**

In [56]:
S2_AC_asset_id = 'users/Hareeshrao-LC60/Atmospheric_Correction2023-09-01'
S2_AC = ee.Image(S2_AC_asset_id)

display(S2_AC)

In [57]:
# Function to calculate methane concentration from Sentinel-2 image
def calculate_methane_enhancement(S2_TOA):
    # Replace 'your_band_name' with the actual band containing atmospheric correction results
    methane_band =S2_TOA.select('B12')
    methane_free_band =S2_TOA.select('B11')
    scale=S2_TOA.projection().nominalScale(),
                        
    # Calculate the ratio of methane band to methane-free band
    methane_ratio = methane_band.divide(methane_free_band)
    
    # Calculate methane enhancement
    methane_enhancement = methane_ratio.subtract(1)
    
    return methane_enhancement
    # Specify the conversion factor if needed
    # methane_concentration = methane_band.multiply(CONVERSION_FACTOR)
    return methane_band

# Replace 'your_sentinel_image' with the actual Sentinel image
sentinel_image = ee.Image(S2_AC)

# Apply atmospheric correction if needed
# atmospheric_corrected_image = sentinel_image.some_atmospheric_correction_function() 

# Mask clouds
#masked_image = mask_clouds(atmospheric_corrected_image)

# Calculate methane enhancement
methane_enhancement_image = calculate_methane_enhancement(sentinel_image)

# Display the methane enhancement on the Folium map
Map = geemap.Map()
Map.centerObject(methane_enhancement_image, zoom=10)
Map.addLayer(methane_enhancement_image, {
    'min': 0.5,
    'max': 0.7,
    'palette': ['purple', 'white', 'red']  # Adjust the palette as needed
}, 'Methane Enhancement')
Map.addLayerControl()
Map

Map(center=[-12.537021337810492, 130.9116975318995], controls=(WidgetControl(options=['position', 'transparent…

# **Add Scaling Factor**

In [58]:
import ee
import numpy as np
from sklearn.linear_model import LinearRegression

# Initialize Google Earth Engine
ee.Initialize()

# Replace 'methane_band_name' and 'methane_free_band_name' with the actual band names
methane_band_name = 'B12'
methane_free_band_name = 'B11'

# Load the atmospherically corrected Sentinel-2 image
S2_ME = ee.Image(S2_AC)

# Select the methane and methane-free bands
methane_band = S2_ME.select(methane_band_name)
methane_free_band = S2_ME.select(methane_free_band_name)

# Get the data as NumPy arrays
methane_data = np.array(methane_band.reduceRegion(reducer=ee.Reducer.toList(), geometry=S2_ME.geometry(), scale=10).get(methane_band_name).getInfo())
methane_free_data = np.array(methane_free_band.reduceRegion(reducer=ee.Reducer.toList(), geometry=S2_ME.geometry(), scale=10).get(methane_free_band_name).getInfo())

# Reshape the data for scikit-learn
X = methane_free_data.reshape(-1, 1)
y = methane_data.reshape(-1, 1)

# Fit a linear regression model
model = LinearRegression().fit(X, y)

# Extract the scaling factor 'c'
scaling_factor_c = model.coef_[0][0]

display(model)
print("Scaling Factor (c):", scaling_factor_c)


LinearRegression()

Scaling Factor (c): 0.011597993134691575


In [62]:
import ee
import folium
import geemap

# Initialize Google Earth Engine
ee.Initialize()

# Replace 'methane_band_name' and 'methane_free_band_name' with the actual band names
methane_band_name = 'B12'
methane_free_band_name = 'B11'

# Load the atmospherically corrected Sentinel-2 image
S2_ME = ee.Image(S2_AC)

# Select the methane and methane-free bands
methane_band = S2_ME.select(methane_band_name)
methane_free_band = S2_ME.select(methane_free_band_name)

# Calculate Methane Enhancement (ME) using the scaling factor
methane_ratio = methane_band.divide(methane_free_band)
                 
methane_minus = methane_ratio.subtract(1)

#methane_enhancement = methane_minus.multiply(scaling_factor_c)

# Display the Methane Enhancement on the Folium map
Map = geemap.Map()
Map.centerObject(S2_ME, zoom=12)

# Add layers to the Folium map
Map.add_ee_layer(methane_minus, {
    'min': 0.1,
    'max': 0.3,  # Adjust the min and max values as needed
    'palette': ['purple', 'white', 'red']
}, 'Methane Enhancement')

# Add layer control panel to the map
Map.addLayerControl()

# Display the map
display(Map)


Map(center=[-12.537021286275174, 130.91169810323842], controls=(WidgetControl(options=['position', 'transparen…

In [60]:
!pip install scikit-learn